# Fetch SEDLS presenter materials from the OSF repository

These scripts were originally created to streamline the process of accessing URL links to presentation resources stored in OSF for incorporation into the Program page on the SEDLS website (e.g., [the links to materials on the 2021 SEDLS Program page](https://se-datalibrarian.github.io/2021/))

In [123]:
import requests
from typing import Dict, List

In [9]:
# Fetch the information providing information on file providers that are
# hosting materials in the OSF repo
sedls_file_providers_fetch = requests.get(sedls_21_files_path)
file_providers = sedls_file_providers_fetch.json()

# List of providers hosting files in OSF repo
file_provider_paths = [
    sedls_21_files_path + provider['attributes']['name'] + '/' for provider in file_providers['data']
]

file_provider_paths

['https://api.osf.io/v2/nodes/pyscb/files/osfstorage/']

In [160]:
def fetch_with_pagination(path: str) -> List[Dict]:
    '''
    Fetch the specified OSF API resource based on the provided "path" and return a list of Dictionary objects containing responses for each page of data from a collection using pagination.

    Parameters
    ----------
    path: str
        The URL path pointing to the OSF API collection of interest

    Returns
    -------
    List[Dict]
        A list of Dictionaries with each item containing the data from one page of data from a collection
    '''

    page_json_list: List[Dict] = [requests.get(path).json()]
    if page_json_list[0]['links']['next'] is not None:
        page_json_list += fetch_with_pagination(page_json_list[0]['links']['next'])
    return page_json_list
    

In [158]:
def fetch_provider_paths(node_id: str) -> List[str]:
    '''
    Gets a ist of the URL paths for file storage providers in an OSF project. Use this list to fetch information on files stored in each provider.

    Parameters
    ----------
    node_id: str
        The node id of the project for which to get the URL paths

    Returns
    -------
    List[str]
        A list of strings of URL paths for file storage providers
    '''

    node_path = f'https://api.osf.io/v2/nodes/{node_id}/files/'
    providers_data = fetch_with_pagination(node_path)
    return [
        path['relationships']['files']['links']['related']['href']
        for provider in providers_data for path in provider['data']
    ]

In [151]:
def fetch_OSF_files(list_of_file_provider_paths: List[str]) -> List[Dict]:
    '''Gets a list of all files in an OSF project
    
    Parameters
    ----------
    list_of_file_provider_paths: List[str]
        A list of URL paths pointing to the storage providers in the OSF repository

    Returns
    -------
    List[Dict]
        A list of dictionaries containing information about each file in the OSF repository
    '''

    # Fetch the contents within each provider's storage (files and folders)
    provider_contents_pages: List[str] = [
        fetch_with_pagination(path) for path in list_of_file_provider_paths
    ]

    # Loop through the contents of each provider and identify if the content
    # type is a file or folder. If it is a file, append it to the files list.
    # If it is a folder, recursively call the "fetch_OSF_files" function until
    # files are returned and concatenate the resulting list to the files list.
    files: List[Dict] = []
    for pages in provider_contents_pages:
        for data in pages:
            for attributes in data['data']:
                if attributes['attributes']['kind'] == 'folder':
                    folder_files = fetch_OSF_files(
                        [
                            attributes['relationships']['files']['links']['related']['href']
                        ]
                    )
                    files += folder_files
                elif attributes['attributes']['kind'] == 'file':
                    files.append(
                        {
                            'id': attributes['attributes']['guid'],
                            'name': attributes['attributes']['name'],
                            'path': attributes['attributes']['materialized_path']
                        }
                    )
    return files

In [162]:
# ID obtained from root page of project: https://osf.io/pyscb/
sedls21_osf_id: str = 'pyscb'
# Fetch the paths for the storage providers in the SEDLS 2021 OSF project
sedls21_providers_paths = fetch_provider_paths(sedls21_osf_id)
# Fetch the files in the SEDLS 2021 OSF project
sedls21_files = fetch_OSF_files(sedls21_providers_paths)

In [163]:
sedls21_files

[{'id': 'gjwy8',
  'name': '7_Bauder_Data-literacy-as-critical-information-literacy.pdf',
  'path': '/Poster presentations/7_Bauder_Data-literacy-as-critical-information-literacy.pdf'},
 {'id': 'pwv2d',
  'name': '1_Smith-and-Davis_Escape-data-horror_Creating-innovative-ways-to-teach-RDM.pdf',
  'path': '/Poster presentations/1_Smith-and-Davis_Escape-data-horror_Creating-innovative-ways-to-teach-RDM.pdf'},
 {'id': 'rtz7y',
  'name': '5_Castro-et-al_dLOC-as-data.pdf',
  'path': '/Poster presentations/5_Castro-et-al_dLOC-as-data.pdf'},
 {'id': 'uc5j2',
  'name': '2_Mckenna-Foster_ETData.pdf',
  'path': '/Poster presentations/2_Mckenna-Foster_ETData.pdf'},
 {'id': 'vgfu8',
  'name': '4_Klein-Carlton_iNaturalist-for-data-literacy.pdf',
  'path': '/Poster presentations/4_Klein-Carlton_iNaturalist-for-data-literacy.pdf'},
 {'id': 'rkn2w',
  'name': '6-Holsapple_Data-sharing-for-Masters-theses.pdf',
  'path': '/Poster presentations/6-Holsapple_Data-sharing-for-Masters-theses.pdf'},
 {'id': 'b